In [4]:
using HTTP, DataFrames, JSON3, Dates

In [60]:
function assemble_url(base_url, data_type; countries = nothing, USstates = nothing)
    base = base_url * "vaccine/"
    if data_type == :trials
        url = base
    elseif data_type == :doses
        doses_url = base * "coverage/"
        
        if countries != nothing
            countries_url = doses_url * "countries/"
            countries == :all ? (url = countries_url) : (url = countries_url * string.(countries))
        
        elseif USstates!= nothing
            USstates_url = doses_url * "states/"
            USstates == :all ? (url = USstates_url) : (url = USstates_url * string.(USstates))
        else
            error("")
        end
        
    else
        error("")
    end
    
    return url
    
end

assemble_url (generic function with 1 method)

In [138]:
function assemble_df(get_data; all_data = true, order_by = :country, vaccine_timeline = :full)
   #order_by can be :state, :country
    
    if all_data
        
        if vaccine_timeline == :simple
            dict = [Dict(order_by => j[string(order_by)], :date => date, :total_cases => cases) 
                for j in JSON3.read(get_data.body) for (date, cases) in j["timeline"]]
            data = DataFrame(dict)
        elseif vaccine_timeline == :full
            raw_df = vcat(DataFrame.(JSON3.read(get_data.body))...)
            timeline = vcat(DataFrame.(raw_df.timeline)...)
            data = hcat(select(raw_df, Not(:timeline)), timeline)    
        
        else 
            error("")
        end
        
    
    elseif all_data == false
                
        if vaccine_timeline == :simple
            raw_data = JSON3.read(get_data.body)
            dict = [Dict(:state => raw_data[string(order_by)], :date => date, :total_cases => cases) 
                     for (date, cases) in raw_data["timeline"]]
            data = DataFrame(dict)     
            
        elseif vaccine_timeline == :full
            raw_df = DataFrame(JSON3.read(get_data.body))
            data = hcat(select(raw_df, Not(:timeline)), DataFrame(raw_df.timeline))
        else
            error()
        end
                
    end
        
    return data
end

assemble_df (generic function with 1 method)

In [ ]:
function opendisease_vaccines(data_type::Val{:trials}; countries = nothing, USstates = nothing, lastdays = nothing, vaccine_timeline = :simple)
    try    
        #vaccine_timeline can be :simple of :full
        base_url = "https://disease.sh/v3/covid-19/"
        url = assemble_url(base_url, data_type)
        get_data = HTTP.get(url)
        data = collect(JSON3.read(get_data.body))
        
        return data  
    catch e
        print(e)
    end 
end

function opendisease_vaccines(data_type::Val{:doses}; countries = nothing, USstates = nothing, lastdays = nothing, vaccine_timeline = :simple)
    try    
        #vaccine_timeline can be :simple of :full
        base_url = "https://disease.sh/v3/covid-19/"            
        vaccine_timeline == :simple ? (fullData = false) : (fullData = true)
        params = Dict("lastdays" =>  lastdays, "fullData" => fullData)
        countries != nothing ? (order_by = :country) : (order_by = :state)
        (countries == :all || USstates == :all) ? (all_data = true) : (all_data = false)
        
        if countries != nothing
            if countries == :all
                url = assemble_url(base_url, data_type, countries = countries)
                get_data = HTTP.get(url, query = params)
                data = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)
                
            else 
                arr_df = []
                for country in 1:length(countries)
                    url = assemble_url(base_url, data_type, countries = countries[country])
                    get_data = HTTP.get(url, query = params)
                    df = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)
                    push!(arr_df, df)
                end
                data = vcat(arr_df...)   
            end
                                 
        elseif USstates != nothing 
            if USstates == :all
                url = assemble_url(base_url, data_type, USstates = USstates)
                get_data = HTTP.get(url, query = params)
                data = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)  
                        
            else
                arr_df = []
                for state in 1:length(USstates)
                    url = assemble_url(base_url, data_type, USstates = USstates[state])
                    get_data = HTTP.get(url, query = params)
                    df = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)
                    push!(arr_df, df)                        
                end
                data = vcat(arr_df...)
            end
            
        else
            error("'vaccine_timeline' must be one of :simple or :full")
        end
        
        return data
        
    catch e
        print(e)
    end
end


function opendisease_vaccines(data_type::Symbol; kwargs...)
    try
        opendisease_vaccines(Val(data_type); kwargs...)
    catch e
       print(e) 
    end 
end

In [169]:
opendisease_vaccines(:trials)

4-element Vector{Pair{Symbol, Any}}:
          :source => "https://www.raps.org/news-and-articles/news-articles/2020/3/covid-19-vaccine-tracker"
 :totalCandidates => "51"
          :phases => JSON3.Object[{
        "phase": "Phase 3",
   "candidates": "10"
}, {
        "phase": "Phase 2/3",
   "candidates": "3"
}, {
        "phase": "Phase 2",
   "candidates": "2"
}, {
        "phase": "Phase 1/2",
   "candidates": "9"
}, {
        "phase": "Phase 1",
   "candidates": "13"
}, {
        "phase": "Pre-clinical",
   "candidates": "14"
}]
            :data => JSON3.Object[{
      "candidate": "BNT162",
      "mechanism": "mRNA-based vaccine",
       "sponsors": [
                     "Pfizer, BioNTech"
                   ],
        "details": "Background: Pfizer and BioNtech are&nbsp;collaborating&nbsp;to develop BNT162, a series of vaccine candidates for COVID-19. BNT162 was initially four candidates developed by BioNTech, two candidates consisting of nucleoside modified mRNA-based (modRN

In [155]:
opendisease_vaccines(:doses, countries = :all, lastdays = 3, vaccine_timeline = :full)

,country,daily,dailyPerMillion,date,total,totalPerHundred
,String,Int64,Int64,String,Int64,Int64
1,Afghanistan,0,0,1/2/22,4674518,0
2,Afghanistan,0,0,1/3/22,4674518,0
3,Afghanistan,0,0,1/4/22,4674518,0
4,Albania,0,0,1/2/22,2316606,0
5,Albania,0,0,1/3/22,2316606,0
6,Albania,0,0,1/4/22,2316606,0
7,Algeria,0,0,1/2/22,12545356,0
8,Algeria,0,0,1/3/22,12545356,0
9,Algeria,0,0,1/4/22,12545356,0


In [156]:
opendisease_vaccines(:doses, countries = :all, lastdays = 3, vaccine_timeline = :simple)

,country,date,total_cases
,String,Symbol,Int64
1,Afghanistan,1/2/22,4674518
2,Afghanistan,1/3/22,4674518
3,Afghanistan,1/4/22,4674518
4,Albania,1/2/22,2316606
5,Albania,1/3/22,2316606
6,Albania,1/4/22,2316606
7,Algeria,1/2/22,12545356
8,Algeria,1/3/22,12545356
9,Algeria,1/4/22,12545356


In [157]:
opendisease_vaccines(:doses, countries = [:MX], lastdays = 3, vaccine_timeline = :full)

,country,daily,dailyPerMillion,totalPerHundred,date,total
,String,Int64,Int64,Int64,String,Int64
1,Mexico,0,0,0,1/2/22,148938454
2,Mexico,0,0,0,1/3/22,148938454
3,Mexico,0,0,0,1/4/22,148938454


In [158]:
opendisease_vaccines(:doses, countries = [:MX], lastdays = 3, vaccine_timeline = :simple)

,state,date,total_cases
,String,Symbol,Int64
1,Mexico,1/2/22,148938454
2,Mexico,1/3/22,148938454
3,Mexico,1/4/22,148938454


In [159]:
opendisease_vaccines(:doses, countries = [:MX, :AU], lastdays = 3, vaccine_timeline = :simple)

,state,date,total_cases
,String,Symbol,Int64
1,Mexico,1/2/22,148938454
2,Mexico,1/3/22,148938454
3,Mexico,1/4/22,148938454
4,Australia,1/2/22,42768270
5,Australia,1/3/22,42812691
6,Australia,1/4/22,42812691


In [160]:
opendisease_vaccines(:doses, countries = [:MX, :AU], lastdays = 3, vaccine_timeline = :full)

,country,daily,dailyPerMillion,totalPerHundred,date,total
,String,Int64,Int64,Int64,String,Int64
1,Mexico,0,0,0,1/2/22,148938454
2,Mexico,0,0,0,1/3/22,148938454
3,Mexico,0,0,0,1/4/22,148938454
4,Australia,80616,3126,165,1/2/22,42768270
5,Australia,82527,3200,166,1/3/22,42812691
6,Australia,0,0,0,1/4/22,42812691


In [161]:
opendisease_vaccines(:doses, USstates = :all, lastdays = 3, vaccine_timeline = :simple)

,state,date,total_cases
,String,Symbol,Int64
1,Alabama,1/2/22,5656460
2,Alabama,1/3/22,5667242
3,Alabama,1/4/22,5678299
4,Alaska,1/2/22,1029700
5,Alaska,1/3/22,1031671
6,Alaska,1/4/22,1030144
7,American Samoa,1/2/22,76482
8,American Samoa,1/3/22,76796
9,American Samoa,1/4/22,78317


In [162]:
opendisease_vaccines(:doses, USstates = :all, lastdays = 3, vaccine_timeline = :full)

,state,daily,dailyPerMillion,date,total,totalPerHundred
,String,Int64,Int64,String,Int64,Int64
1,Alabama,10762,2195,1/2/22,5656460,0
2,Alabama,10782,2199,1/3/22,5667242,0
3,Alabama,10478,2137,1/4/22,5678299,115
4,Alaska,1820,2488,1/2/22,1029700,0
5,Alaska,1971,2694,1/3/22,1031671,0
6,Alaska,1293,1767,1/4/22,1030144,140
7,American Samoa,268,5655,1/2/22,76482,0
8,American Samoa,314,6626,1/3/22,76796,0
9,American Samoa,357,7533,1/4/22,78317,165


In [163]:
opendisease_vaccines(:doses, USstates = [:illinois], lastdays = 3, vaccine_timeline = :simple)

,state,date,total_cases
,String,Symbol,Int64
1,Illinois,1/2/22,19872539
2,Illinois,1/3/22,19919576
3,Illinois,1/4/22,20006703


In [164]:
opendisease_vaccines(:doses, USstates = [:illinois], lastdays = 3, vaccine_timeline = :full)

,state,daily,dailyPerMillion,totalPerHundred,date,total
,String,Int64,Int64,Int64,String,Int64
1,Illinois,45128,3561,0,1/2/22,19872539
2,Illinois,47037,3712,0,1/3/22,19919576
3,Illinois,48946,3863,157,1/4/22,20006703


In [165]:
opendisease_vaccines(:doses, USstates = [:illinois, :texas], lastdays = 3, vaccine_timeline = :simple)

,state,date,total_cases
,String,Symbol,Int64
1,Illinois,1/2/22,19872539
2,Illinois,1/3/22,19919576
3,Illinois,1/4/22,20006703
4,Texas,1/2/22,40874750
5,Texas,1/3/22,40965089
6,Texas,1/4/22,40980790


In [166]:
opendisease_vaccines(:doses, USstates = [:illinois, :texas], lastdays = 3, vaccine_timeline = :full)

,state,daily,dailyPerMillion,totalPerHundred,date,total
,String,Int64,Int64,Int64,String,Int64
1,Illinois,45128,3561,0,1/2/22,19872539
2,Illinois,47037,3712,0,1/3/22,19919576
3,Illinois,48946,3863,157,1/4/22,20006703
4,Texas,88949,3068,0,1/2/22,40874750
5,Texas,90339,3116,0,1/3/22,40965089
6,Texas,89415,3084,141,1/4/22,40980790
